## Import Packages

In [1]:
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn import metrics
from joblib import dump, load

## Set seed

In [2]:
random_seed = 8022022 # or any of your favorite number 
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

In [3]:
outcome = "was_early_preterm"
combine_outcome = False

## Read dataset

### read metadata

In [4]:
metadata = pd.read_csv('data/raw_combine/comb_meta_alpha.csv', delimiter=',',index_col = 'specimen').sort_index()

In [5]:
print(metadata.shape)
metadata.head(5)

(2461, 14)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,inv_simpson,bwpd,phylo_entropy,quadratic,unrooted_pd,rooted_pd
specimen,,,,,,,,,,,,,,
A00001-05,A00001,A,False,False,38,33,III,1.00000,1.00000,0.00000,0.00000,0.000000,0.00000,2.53935
A00002-01,A00002,A,False,False,40,38,III,1.96362,1.81277,2.62894,1.31887,0.876314,3.94341,4.14816
A00003-02,A00003,A,False,False,40,30,II,1.00000,1.00000,0.00000,0.00000,0.000000,0.00000,2.62632
A00004-08,A00004,A,False,False,40,27,III,1.00000,1.00000,0.00000,0.00000,0.000000,0.00000,1.83870
A00004-12,A00004,A,False,False,40,29,III,6.94884,4.07385,2.78896,3.13422,1.219900,15.51850,15.58460


### read taxonomy

In [6]:
taxonomy1 = pd.read_csv('data/raw_combine/comb_taxonomy_relabd.family.csv', delimiter=',',index_col = 'specimen')
tax_feature1 = list(pd.read_csv('data/selected_feature/tax_family_early_preterm.csv', delimiter=',').feature_selected)
taxonomy_selected1 = taxonomy1[tax_feature1].sort_index()
tax_old_name1 = list(taxonomy_selected1.columns)
tax_new_name1 = [item + '_tax1' for item in tax_old_name1]
tax_renamedict1 = dict(zip(tax_old_name1,tax_new_name1))

taxonomy_selected1 = taxonomy_selected1.rename(columns = tax_renamedict1)

In [7]:
taxonomy2 = pd.read_csv('data/raw_combine/comb_taxonomy_relabd.genus.csv', delimiter=',',index_col = 'specimen')
tax_feature2 = list(pd.read_csv('data/selected_feature/tax_genus_early_preterm.csv', delimiter=',').feature_selected)
taxonomy_selected2 = taxonomy2[tax_feature2].sort_index()
tax_old_name2 = list(taxonomy_selected2.columns)
tax_new_name2 = [item + '_tax2' for item in tax_old_name2]
tax_renamedict2 = dict(zip(tax_old_name2,tax_new_name2))

taxonomy_selected2 = taxonomy_selected2.rename(columns = tax_renamedict2)

In [8]:
taxonomy3 = pd.read_csv('data/raw_combine/comb_taxonomy_relabd.species.csv', delimiter=',',index_col = 'specimen')
tax_feature3 = list(pd.read_csv('data/selected_feature/tax_species_early_preterm.csv', delimiter=',').feature_selected)
taxonomy_selected3 = taxonomy3[tax_feature3].sort_index()
tax_old_name3 = list(taxonomy_selected3.columns)
tax_new_name3 = [item + '_tax3' for item in tax_old_name3]
tax_renamedict3 = dict(zip(tax_old_name3,tax_new_name3))

taxonomy_selected3 = taxonomy_selected3.rename(columns = tax_renamedict3)

In [9]:
taxonomy_selected = pd.concat([taxonomy_selected1,taxonomy_selected2,taxonomy_selected3], axis = 1).copy()

In [10]:
taxonomy_selected

,Prevotellaceae_tax1,Fusobacteriaceae_tax1,Veillonellaceae_tax1,Leptotrichiaceae_tax1,Ruminococcaceae_tax1,Mycoplasmataceae_tax1,Staphylococcaceae_tax1,unclassified.Clostridiales_tax1,Bacteroidales_tax1,unclassified.Corynebacteriales_tax1,...,Sneathia.amnii_tax3,Prevotella.disiens_tax3,Pseudomonas.fluorescens.veronii_tax3,Anaerococcus.lactolyticus_tax3,Aerococcus.christensenii_tax3,Atopobium.vaginae_tax3,Bacteroidales_tax3,Dialister.micraerophilus_tax3,Lactobacillus.jensenii_tax3,Lactobacillus.crispatus_tax3
specimen,,,,,,,,,,,,,,,,,,,,,
A00001-05,0.113386,0.00105,0.008924,0.000000,0.0,0.003150,0.003675,0.022572,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
A00002-01,0.011649,0.00000,0.001226,0.028817,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000613,0.000000
A00003-02,0.011874,0.00000,0.000216,0.000216,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.011442,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
A00004-08,0.031860,0.00000,0.000000,0.000000,0.0,0.002569,0.000514,0.000514,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
A00004-12,0.143407,0.00000,0.000733,0.000183,0.0,0.003846,0.000000,0.002564,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.002015,0.0,0.000549,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J00111-01,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.004992
J00112-01,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.141649,0.842958
J00113-01,0.000631,0.00000,0.000000,0.472547,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.472547,0.000000,0.0,0.0,0.0,0.005049,0.0,0.000000,0.001736,0.004733


### read phylotype

In [11]:
phylotype1 = pd.read_csv('data/raw_combine/comb_phylotype_relabd.1e_1.csv', delimiter=',',index_col = 'specimen')
phylo_feature1 = list(pd.read_csv('data/selected_feature/phylo_.1_early_preterm.csv', delimiter=',').feature_selected)
phylotype_selected1 = phylotype1[phylo_feature1].sort_index()
phylo_old_name1 = list(phylotype_selected1.columns)
phylo_new_name1 = [item + '_phylo1' for item in phylo_old_name1]
phylo_renamedict1 = dict(zip(phylo_old_name1,phylo_new_name1))

phylotype_selected1 = phylotype_selected1.rename(columns = phylo_renamedict1)

In [12]:
phylotype2 = pd.read_csv('data/raw_combine/comb_phylotype_relabd.5e_1.csv', delimiter=',',index_col = 'specimen')
phylo_feature2 = list(pd.read_csv('data/selected_feature/phylo_.5_early_preterm.csv', delimiter=',').feature_selected)
phylotype_selected2 = phylotype2[phylo_feature2].sort_index()
phylo_old_name2 = list(phylotype_selected2.columns)
phylo_new_name2 = [item + '_phylo2' for item in phylo_old_name2]
phylo_renamedict2 = dict(zip(phylo_old_name2,phylo_new_name2))

phylotype_selected2 = phylotype_selected2.rename(columns = phylo_renamedict2)

In [13]:
phylotype3 = pd.read_csv('data/raw_combine/comb_phylotype_relabd.1e0.csv', delimiter=',',index_col = 'specimen')
phylo_feature3 = list(pd.read_csv('data/selected_feature/phylo_1_early_preterm.csv', delimiter=',').feature_selected)
phylotype_selected3 = phylotype3[phylo_feature3].sort_index()
phylo_old_name3 = list(phylotype_selected3.columns)
phylo_new_name3 = [item + '_phylo3' for item in phylo_old_name3]
phylo_renamedict3 = dict(zip(phylo_old_name3,phylo_new_name3))

phylotype_selected3 = phylotype_selected3.rename(columns = phylo_renamedict3)

In [14]:
phylotype_selected = pd.concat([phylotype_selected1,phylotype_selected2,phylotype_selected3], axis = 1).copy()

In [15]:
phylotype_selected

,pt__00019_phylo1,pt__00056_phylo1,pt__00090_phylo1,pt__00042_phylo1,pt__00034_phylo1,pt__00010_phylo1,pt__00046_phylo1,pt__00013_phylo1,pt__00006_phylo1,pt__00008_phylo1,...,pt__00007_phylo3,pt__00009_phylo3,pt__00027_phylo3,pt__00030_phylo3,pt__00021_phylo3,pt__00032_phylo3,pt__00010_phylo3,pt__00011_phylo3,pt__00006_phylo3,pt__00001_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00001-05,0.000000,0.0,0.0,0.022572,0.000000,0.0,0.0,0.002625,0.0,0.005774,...,0.106037,0.009449,0.0,0.00105,0.000000,0.006299,0.0,0.0,0.000000,0.797900
A00002-01,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000613,...,0.000613,0.001839,0.0,0.00000,0.028817,0.009810,0.0,0.0,0.000000,0.805641
A00003-02,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000216,0.001079,0.0,0.00000,0.000216,0.003022,0.0,0.0,0.000000,0.963299
A00004-08,0.000000,0.0,0.0,0.000514,0.000000,0.0,0.0,0.002569,0.0,0.000000,...,0.027235,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.927544
A00004-12,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.003846,0.0,0.000000,...,0.142857,0.000916,0.0,0.00000,0.000183,0.003846,0.0,0.0,0.000366,0.806593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J00111-01,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,1.000000
J00112-01,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.999644
J00113-01,0.000631,0.0,0.0,0.000000,0.002682,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.483433


### combine data

In [16]:
mydata = pd.concat([metadata,taxonomy_selected,phylotype_selected], axis = 1).copy()

In [17]:
print(mydata.shape)
mydata.head(5)

(2461, 115)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,inv_simpson,bwpd,...,pt__00007_phylo3,pt__00009_phylo3,pt__00027_phylo3,pt__00030_phylo3,pt__00021_phylo3,pt__00032_phylo3,pt__00010_phylo3,pt__00011_phylo3,pt__00006_phylo3,pt__00001_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00001-05,A00001,A,False,False,38,33,III,1.00000,1.00000,0.00000,...,0.106037,0.009449,0.0,0.00105,0.000000,0.006299,0.0,0.0,0.000000,0.797900
A00002-01,A00002,A,False,False,40,38,III,1.96362,1.81277,2.62894,...,0.000613,0.001839,0.0,0.00000,0.028817,0.009810,0.0,0.0,0.000000,0.805641
A00003-02,A00003,A,False,False,40,30,II,1.00000,1.00000,0.00000,...,0.000216,0.001079,0.0,0.00000,0.000216,0.003022,0.0,0.0,0.000000,0.963299
A00004-08,A00004,A,False,False,40,27,III,1.00000,1.00000,0.00000,...,0.027235,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.927544
A00004-12,A00004,A,False,False,40,29,III,6.94884,4.07385,2.78896,...,0.142857,0.000916,0.0,0.00000,0.000183,0.003846,0.0,0.0,0.000366,0.806593


### change data type

In [19]:
mydata["project"] = mydata["project"].astype('category')
mydata["CST"] = mydata["CST"].astype('category')
mydata['was_preterm'] = mydata['was_preterm'].astype('int64')
mydata['was_early_preterm'] = mydata['was_early_preterm'].astype('int64')

mydata.dtypes

participant_id         object
project              category
was_preterm             int64
was_early_preterm       int64
delivery_wk             int64
                       ...   
pt__00032_phylo3      float64
pt__00010_phylo3      float64
pt__00011_phylo3      float64
pt__00006_phylo3      float64
pt__00001_phylo3      float64
Length: 115, dtype: object

### subset data by outcome

In [41]:
# only keep the rows with collect_wk < 32 for early_preterm task
mydata_early_preterm_t = mydata.loc[mydata['collect_wk']<=28,].copy().sort_index(axis = 0)

mydata_early_preterm = mydata_early_preterm_t
# mydata_early_preterm = mydata_early_preterm_t.loc[mydata_early_preterm_t['project']!= 'I',].copy().sort_index(axis = 0)
print(mydata_early_preterm.shape)

mydata_early_preterm.head(5)

(1765, 115)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,inv_simpson,bwpd,...,pt__00007_phylo3,pt__00009_phylo3,pt__00027_phylo3,pt__00030_phylo3,pt__00021_phylo3,pt__00032_phylo3,pt__00010_phylo3,pt__00011_phylo3,pt__00006_phylo3,pt__00001_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00004-08,A00004,A,0,0,40,27,III,1.00000,1.00000,0.00000,...,0.027235,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.927544
A00005-02,A00005,A,0,0,41,26,II,1.85087,1.79889,3.25318,...,0.000476,0.001905,0.001905,0.0,0.0,0.003810,0.0,0.0,0.0,0.755714
A00008-02,A00008,A,1,0,35,17,IV-B,1.00000,1.00000,0.00000,...,0.000000,0.097744,0.000000,0.0,0.0,0.003008,0.0,0.0,0.0,0.156391
A00008-08,A00008,A,1,0,35,26,IV-B,2.34379,2.12473,2.41737,...,0.000000,0.033789,0.000279,0.0,0.0,0.000279,0.0,0.0,0.0,0.134320
A00009-04,A00009,A,0,0,37,25,IV-C,1.58125,1.49373,2.07183,...,0.000000,0.000442,0.000442,0.0,0.0,0.000000,0.0,0.0,0.0,0.982092


In [42]:
mydata_early_preterm_ind = mydata_early_preterm.groupby('participant_id').first().copy()

mydata_early_preterm_ind.shape

(1012, 114)

### split train validation id set

In [43]:
test_id = list(mydata_early_preterm_ind.groupby(outcome).sample(frac = 0.2, random_state = 200).sort_index(axis = 0).index)

# train_val_id = list(mydata_early_preterm_ind.loc[~mydata_early_preterm_ind.index.isin(train_id)].groupby('was_early_preterm').\
#             sample(frac = 0.5, random_state = 200).sort_index(axis = 0).index)

train_val_id = list(np.setdiff1d(list(mydata_early_preterm_ind.index),test_id))

print(len(test_id))
print(len(train_val_id))


202
810


### obtain train validation dataset

In [44]:
test_data = mydata_early_preterm[mydata_early_preterm['participant_id'].isin(test_id)]

print(len(test_id))
print(test_data.shape)
test_data.head(5)

202
(387, 115)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,inv_simpson,bwpd,...,pt__00007_phylo3,pt__00009_phylo3,pt__00027_phylo3,pt__00030_phylo3,pt__00021_phylo3,pt__00032_phylo3,pt__00010_phylo3,pt__00011_phylo3,pt__00006_phylo3,pt__00001_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00014-01,A00014,A,0,0,41,13,IV-C,1.0000,1.00000,0.00000,...,0.000213,0.015316,0.000000,0.001489,0.000000,0.012338,0.0,0.0,0.000000,0.850032
A00014-02,A00014,A,0,0,41,14,IV-C,1.0000,1.00000,0.00000,...,0.000163,0.006193,0.000163,0.000489,0.000000,0.004074,0.0,0.0,0.000000,0.943123
A00014-04,A00014,A,0,0,41,16,IV-C,3.5433,2.88222,3.26851,...,0.000418,0.009188,0.001253,0.001670,0.000209,0.014617,0.0,0.0,0.000418,0.873042
A00014-11,A00014,A,0,0,41,23,III,1.0000,1.00000,0.00000,...,0.000289,0.000000,0.000000,0.000000,0.000000,0.000144,0.0,0.0,0.000000,0.983239
A00014-13,A00014,A,0,0,41,25,IV-C,1.0000,1.00000,0.00000,...,0.000555,0.046103,0.001481,0.019071,0.000000,0.027402,0.0,0.0,0.000000,0.604518


In [45]:
train_val_data = mydata_early_preterm[mydata_early_preterm['participant_id'].isin(train_val_id)]

print(len(train_val_id))
print(train_val_data.shape)
train_val_data.head(5)

810
(1378, 115)


,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,inv_simpson,bwpd,...,pt__00007_phylo3,pt__00009_phylo3,pt__00027_phylo3,pt__00030_phylo3,pt__00021_phylo3,pt__00032_phylo3,pt__00010_phylo3,pt__00011_phylo3,pt__00006_phylo3,pt__00001_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00004-08,A00004,A,0,0,40,27,III,1.00000,1.00000,0.00000,...,0.027235,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.927544
A00005-02,A00005,A,0,0,41,26,II,1.85087,1.79889,3.25318,...,0.000476,0.001905,0.001905,0.0,0.0,0.003810,0.0,0.0,0.0,0.755714
A00008-02,A00008,A,1,0,35,17,IV-B,1.00000,1.00000,0.00000,...,0.000000,0.097744,0.000000,0.0,0.0,0.003008,0.0,0.0,0.0,0.156391
A00008-08,A00008,A,1,0,35,26,IV-B,2.34379,2.12473,2.41737,...,0.000000,0.033789,0.000279,0.0,0.0,0.000279,0.0,0.0,0.0,0.134320
A00009-04,A00009,A,0,0,37,25,IV-C,1.58125,1.49373,2.07183,...,0.000000,0.000442,0.000442,0.0,0.0,0.000000,0.0,0.0,0.0,0.982092


In [46]:
def tensor_generator(data, outcome = outcome, combine_outcome = combine_outcome):
    X_data = data[data.columns[5:]].copy()
    X_data['CST'] = X_data['CST'].cat.codes
    X_feature = X_data.to_numpy().astype('float32')
    X_group = data['participant_id'].astype('category').cat.codes.values.reshape(-1,1)
    
    if combine_outcome:
        input_X = torch.from_numpy(np.hstack((X_feature,X_group)))
        y_data = data.groupby('participant_id').first()
        y = y_data[outcome].values.flatten()
        input_y = torch.from_numpy(np.vstack((y,1-y)).T.astype('float32'))
    else:
        input_X = torch.from_numpy(X_feature)
        y = data[outcome].values.flatten()
        input_y = torch.from_numpy(np.vstack((y,1-y)).T.astype('float32'))
    
    
    return input_X, input_y

    

In [47]:
test_data.head(5)

,participant_id,project,was_preterm,was_early_preterm,delivery_wk,collect_wk,CST,shannon,inv_simpson,bwpd,...,pt__00007_phylo3,pt__00009_phylo3,pt__00027_phylo3,pt__00030_phylo3,pt__00021_phylo3,pt__00032_phylo3,pt__00010_phylo3,pt__00011_phylo3,pt__00006_phylo3,pt__00001_phylo3
specimen,,,,,,,,,,,,,,,,,,,,,
A00014-01,A00014,A,0,0,41,13,IV-C,1.0000,1.00000,0.00000,...,0.000213,0.015316,0.000000,0.001489,0.000000,0.012338,0.0,0.0,0.000000,0.850032
A00014-02,A00014,A,0,0,41,14,IV-C,1.0000,1.00000,0.00000,...,0.000163,0.006193,0.000163,0.000489,0.000000,0.004074,0.0,0.0,0.000000,0.943123
A00014-04,A00014,A,0,0,41,16,IV-C,3.5433,2.88222,3.26851,...,0.000418,0.009188,0.001253,0.001670,0.000209,0.014617,0.0,0.0,0.000418,0.873042
A00014-11,A00014,A,0,0,41,23,III,1.0000,1.00000,0.00000,...,0.000289,0.000000,0.000000,0.000000,0.000000,0.000144,0.0,0.0,0.000000,0.983239
A00014-13,A00014,A,0,0,41,25,IV-C,1.0000,1.00000,0.00000,...,0.000555,0.046103,0.001481,0.019071,0.000000,0.027402,0.0,0.0,0.000000,0.604518


In [48]:
mydata_early_preterm[mydata_early_preterm.columns[5:]].columns

Index(['collect_wk', 'CST', 'shannon', 'inv_simpson', 'bwpd', 'phylo_entropy',
       'quadratic', 'unrooted_pd', 'rooted_pd', 'Prevotellaceae_tax1',
       ...
       'pt__00007_phylo3', 'pt__00009_phylo3', 'pt__00027_phylo3',
       'pt__00030_phylo3', 'pt__00021_phylo3', 'pt__00032_phylo3',
       'pt__00010_phylo3', 'pt__00011_phylo3', 'pt__00006_phylo3',
       'pt__00001_phylo3'],
      dtype='object', length=110)

In [49]:
# Define RNN model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1,hidden_dim2,
                 layer3, hidden_dim3,
                 drop_out,combine_outcome):
        

        #inherit from super class
        super(MLP, self).__init__()
        
        #define combine 
        
        self.combine = combine_outcome
        self.layer3 = layer3
        
        #define layers
        if layer3:
            self.fc1 = nn.Linear(input_dim, hidden_dim1)
            self.fc2 = nn.Linear(hidden_dim1,hidden_dim2)
            self.fc3 = nn.Linear(hidden_dim2,hidden_dim3)
            self.fc4 = nn.Linear(hidden_dim3,2)
        else:
            self.fc1 = nn.Linear(input_dim, hidden_dim1)
            self.fc2 = nn.Linear(hidden_dim1,hidden_dim2)
            self.fc3 = nn.Linear(hidden_dim2,2)

        self.dropout = nn.Dropout(drop_out)

        
    def forward(self, x):
        
        if self.combine:
        
            X_feature = x[:,:-1]
            X_group = x[:,-1].long()
        
            X_feature = torch.tanh(self.fc1(X_feature))
            X_feature = self.dropout(X_feature)
            X_feature = F.relu(self.fc2(X_feature))
            X_feature = self.dropout(X_feature)
            X_feature = torch.tanh(self.fc3(X_feature))
            
            if self.layer3:
                X_feature = self.dropout(X_feature)
                X_feature = torch.tanh(self.fc4(X_feature))

            X_feature = F.softmax(X_feature, dim = 1)

        
            M = torch.zeros(X_group.max()+1, len(X_feature))
            M[X_group, torch.arange(len(X_feature))] = 1
            M = F.normalize(M, p=1, dim=1)
            X_feature = torch.mm(M, X_feature)
            
        else: 
            
            X_feature = x
            
            X_feature = torch.tanh(self.fc1(X_feature))
            X_feature = self.dropout(X_feature)
            X_feature = F.relu(self.fc2(X_feature))
            X_feature = self.dropout(X_feature)
            X_feature = torch.tanh(self.fc3(X_feature))
            
            if self.layer3:
                X_feature = self.dropout(X_feature)
                X_feature = torch.tanh(self.fc4(X_feature))

            X_feature = F.softmax(X_feature, dim = 1)
        
        return X_feature
    

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01) 


In [50]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [51]:
def indloader(data, seed, outcome = outcome):
    data_ind = data.groupby('participant_id').first().copy()
    id_1 = list(data_ind.groupby(outcome).sample(frac = 0.5, random_state = seed).sort_index(axis = 0).index)
    id_2 = list(np.setdiff1d(list(data_ind.index),id_1))
    
    return [id_1,id_2]
    

In [52]:
def train_epoch(train_data, val_data, learn_rate, \
                hidden_dim1 = 10,hidden_dim2 = 10,layer3 = False,\
                hidden_dim3 = 0,drop_out = 0.1, \
                device = device, EPOCHS = 200, patience = 20,combine_outcome = combine_outcome,outcome = outcome):
    
    val_X, val_y = tensor_generator(val_data, outcome = outcome, combine_outcome = combine_outcome)
    
    
    #input_dim
    if combine_outcome:
        input_dim = val_X.shape[1]-1
    else:
        input_dim = val_X.shape[1]
    

    model = MLP(input_dim, hidden_dim1,hidden_dim2,layer3, hidden_dim3,
                 drop_out,combine_outcome)

        
    model.to(device)
    
    model.apply(init_weights)
    # loss criterion and optimizer

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learn_rate)
    
    train_loss = []
    val_loss = []
    # train model
    
    model.train()
    
    last_loss = 1000
    trigger_times = 0
    #Start training loop
    
    for epoch in range(1, EPOCHS + 1):
        
        batch_train_losses = []
        ind_loader = indloader(train_data, seed = 980 + epoch, outcome = outcome)
        
        
        for ind in ind_loader:
            
            train_batch = train_data[train_data['participant_id'].isin(ind)]
            
            x, label = tensor_generator(train_batch,outcome = outcome, combine_outcome = combine_outcome)
            
            x, label = x.to(device), label.to(device)
            
            model.zero_grad()
        
            predictions = model(x)
            
            predictions = predictions.to(device)
        
            loss = criterion(predictions, label)
            
            batch_train_losses.append(loss.detach().numpy())
        
            # backpropagation
            loss.backward() 
            # Updates the weights accordingly
            optimizer.step()
        
        train_loss.append(np.mean(batch_train_losses))
        
        with torch.no_grad():
            
            model.eval()
            
            val_X, val_y = val_X.to(device), val_y.to(device)
                
            predictions_val = model(val_X)
                
            predictions_val = predictions_val.to(device)
                
            loss_val = criterion(predictions_val, val_y)
            
            val_loss.append(loss_val.detach().numpy())
            

        current_loss = val_loss[epoch-1]
        
        
        if current_loss >= last_loss:
            trigger_times += 1
            
            if trigger_times >= patience:
                print ('Early stopping at {}'.format(epoch))
                return model,train_loss,val_loss
        else:
            tigger_times = 0
        
        last_loss = current_loss
        
        if epoch%10 == 0:
            print('Epoch: {}/{}.............'.format(epoch, EPOCHS), end = ' ')
            print("Train Loss: {:.4f}".format(train_loss[epoch-1]), end = ' ')
            print("Validation Loss: {:.4f}".format(val_loss[epoch-1]))
            
        
    return model,train_loss,val_loss


smaller batch size (2^)
epoch (early stopping)
residual net

In [53]:
def test_metrics(model, val_data, outcome = outcome, combine_outcome = combine_outcome):
    
    val_X = tensor_generator(val_data, outcome = outcome, combine_outcome = combine_outcome)[0]
    
    if combine_outcome:
        val_y = val_data.groupby('participant_id').first()[outcome].values.flatten()
    else:
        val_y = val_data[outcome].values.flatten()
    
    model.eval()
    

    out = model(val_X)
    predicted_props = out[:,0].detach().numpy()
    predicted_labels = 1*(predicted_props >=0.5)
    
    
    acc = metrics.accuracy_score(val_y, predicted_labels, normalize=False) / float(val_y.size)
    confusion = metrics.confusion_matrix(val_y, predicted_labels)
    

    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    
    specificity = TN / (TN + FP)
    sensitivity = TP / (TP + FN)
    precision = TP/(TP + FP)
    
    auc = metrics.roc_auc_score(val_y, predicted_props)
    
    if combine_outcome:
    
        result_tab = pd.DataFrame(data = {'participant_id': list(val_data.groupby('participant_id').first().index),
                                         'labels':list(val_y),
                                         'predicted_labels':list(predicted_labels),
                                         'predicted_probs':list(predicted_props)})
    else:
        result_tab = pd.DataFrame(data = {'participant_id': list(val_data.index),
                                         'labels':list(val_y),
                                         'predicted_labels':list(predicted_labels),
                                         'predicted_probs':list(predicted_props)})
    

    
    return acc,sensitivity,specificity,auc,precision,confusion,result_tab


In [54]:
import itertools

a = [[0.001,0.05], \
    [16,32,64], \
    [16,32,64], \
    [False,True], \
    [16,32,64], \
    [0.05,0.1,0.2]]



parameter_space = list(itertools.product(*a))

len(parameter_space)



324

In [55]:
def parameter_tuning(train_val_data, parameter_space,cv_time =5,seed = 999,outcome = outcome, combine_outcome = combine_outcome):
    
    best_parameter = tuple()
    best_auc = 0
    best_model = None
    
    train_val_data_ind = train_val_data.groupby('participant_id').first().copy()
    
    for param in parameter_space:
        
        auc_cv = []
        
        for i in range(cv_time):
            
            train_id = list(train_val_data_ind.groupby(outcome).sample(frac = 0.75, random_state = seed + i).sort_index(axis = 0).index)
            val_id =  list(np.setdiff1d(list(train_val_data_ind.index),train_id))
            
            train_data = train_val_data[train_val_data['participant_id'].isin(train_id)]
            val_data = train_val_data[train_val_data['participant_id'].isin(val_id)]
        
            model = train_epoch(train_data,val_data,learn_rate = param[0], \
                               hidden_dim1= param[1], hidden_dim2= param[2], \
                               layer3 = param[3], hidden_dim3= param[4],drop_out= param[5], device = device)[0]
        
            auc = test_metrics(model, val_data, outcome = outcome, combine_outcome = combine_outcome)[3]
            
            auc_cv.append(auc)
        
        auc_ave = np.mean(auc_cv)
        
        if auc_ave >= best_auc:
            best_auc = auc_ave
            best_parameter = param
            best_model = model
            
    return best_parameter, best_model,best_auc
        
    

    

In [56]:
best_parameter, best_model,best_auc = parameter_tuning(train_val_data,parameter_space,cv_time =3,seed = 999,outcome = outcome)

Epoch: 10/200............. Train Loss: 0.6584 Validation Loss: 0.6527
Epoch: 20/200............. Train Loss: 0.6193 Validation Loss: 0.6068
Epoch: 30/200............. Train Loss: 0.5711 Validation Loss: 0.5529
Epoch: 40/200............. Train Loss: 0.5463 Validation Loss: 0.5255
Epoch: 50/200............. Train Loss: 0.5371 Validation Loss: 0.5153
Epoch: 60/200............. Train Loss: 0.5340 Validation Loss: 0.5110
Epoch: 70/200............. Train Loss: 0.5316 Validation Loss: 0.5088
Epoch: 80/200............. Train Loss: 0.5305 Validation Loss: 0.5074
Epoch: 90/200............. Train Loss: 0.5306 Validation Loss: 0.5065
Epoch: 100/200............. Train Loss: 0.5293 Validation Loss: 0.5059
Epoch: 110/200............. Train Loss: 0.5290 Validation Loss: 0.5054
Epoch: 120/200............. Train Loss: 0.5282 Validation Loss: 0.5051
Epoch: 130/200............. Train Loss: 0.5280 Validation Loss: 0.5048
Epoch: 140/200............. Train Loss: 0.5287 Validation Loss: 0.5046
Epoch: 150/200.

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_9940\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/200............. Train Loss: 0.6609 Validation Loss: 0.6586
Epoch: 20/200............. Train Loss: 0.6140 Validation Loss: 0.6089
Epoch: 30/200............. Train Loss: 0.5613 Validation Loss: 0.5570
Epoch: 40/200............. Train Loss: 0.5386 Validation Loss: 0.5352
Epoch: 50/200............. Train Loss: 0.5307 Validation Loss: 0.5279
Epoch: 60/200............. Train Loss: 0.5277 Validation Loss: 0.5248
Epoch: 70/200............. Train Loss: 0.5266 Validation Loss: 0.5232
Epoch: 80/200............. Train Loss: 0.5251 Validation Loss: 0.5222
Epoch: 90/200............. Train Loss: 0.5247 Validation Loss: 0.5216
Epoch: 100/200............. Train Loss: 0.5240 Validation Loss: 0.5211
Epoch: 110/200............. Train Loss: 0.5227 Validation Loss: 0.5207
Epoch: 120/200............. Train Loss: 0.5236 Validation Loss: 0.5205
Epoch: 130/200............. Train Loss: 0.5227 Validation Loss: 0.5203
Epoch: 140/200............. Train Loss: 0.5235 Validation Loss: 0.5201
Epoch: 150/200.

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_9940\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/200............. Train Loss: 0.6373 Validation Loss: 0.6356
Epoch: 20/200............. Train Loss: 0.5882 Validation Loss: 0.5851
Epoch: 30/200............. Train Loss: 0.5461 Validation Loss: 0.5452
Epoch: 40/200............. Train Loss: 0.5322 Validation Loss: 0.5319
Epoch: 50/200............. Train Loss: 0.5266 Validation Loss: 0.5277
Epoch: 60/200............. Train Loss: 0.5245 Validation Loss: 0.5258
Epoch: 70/200............. Train Loss: 0.5241 Validation Loss: 0.5249
Epoch: 80/200............. Train Loss: 0.5233 Validation Loss: 0.5242
Epoch: 90/200............. Train Loss: 0.5233 Validation Loss: 0.5238
Epoch: 100/200............. Train Loss: 0.5223 Validation Loss: 0.5235
Epoch: 110/200............. Train Loss: 0.5219 Validation Loss: 0.5233
Epoch: 120/200............. Train Loss: 0.5218 Validation Loss: 0.5231
Epoch: 130/200............. Train Loss: 0.5218 Validation Loss: 0.5230
Epoch: 140/200............. Train Loss: 0.5215 Validation Loss: 0.5229
Epoch: 150/200.

...

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_9940\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/200............. Train Loss: 0.5211 Validation Loss: 0.5190
Epoch: 20/200............. Train Loss: 0.5220 Validation Loss: 0.5190
Early stopping at 24


C:\Users\sxiao15\AppData\Local\Temp\ipykernel_9940\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Epoch: 10/200............. Train Loss: 0.5203 Validation Loss: 0.5222
Epoch: 20/200............. Train Loss: 0.5209 Validation Loss: 0.5222
Early stopping at 24


C:\Users\sxiao15\AppData\Local\Temp\ipykernel_9940\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


In [57]:
best_auc

0.522636815920398

In [151]:
best_parameter

(0.001, 32, 32, False, 32, 0.1)

In [58]:
test_acc,test_sensitivity,test_specificity, \
test_auc,test_precision,test_confusion,test_result_tab = \
test_metrics(best_model, test_data, outcome = outcome)


C:\Users\sxiao15\AppData\Local\Temp\ipykernel_9940\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


In [153]:
test_result_tab

,participant_id,labels,predicted_labels,predicted_probs
0,A00004-08,0,0,0.163488
1,A00004-12,0,0,0.146660
2,A00004-13,0,0,0.145323
3,A00010-01,0,0,0.261438
4,A00010-02,0,0,0.416276
...,...,...,...,...
377,J00093-01,0,0,0.175278
378,J00100-01,0,0,0.122201
379,J00102-01,0,0,0.179555
380,J00105-01,0,0,0.196612


In [63]:
best_model

MLP(
  (fc1): Linear(in_features=41, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=2, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [64]:
dump(best_model,'Submission/test_submission.save')

['Submission/test_submission.save']

In [59]:
test_result_tab.to_csv('test_error_analysis_early.csv')

In [60]:
test_metrics(best_model, train_val_data, outcome = outcome)[6].to_csv('train_val_error_analysis_early.csv')

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_9940\871753563.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


In [ ]:
# def project_cross_val(features,train_ratio = 0.7, learn_rate = 0.01,hidden_dim = 18, n_layers = 1, device = device, \
#                       batch_size = 1000,drop_prob = 0.2, EPOCHS = 1000, output_dim = 2, method = "RNN", \
#                       out_var = "was_early_preterm",label_smooth= True, multi_outcome= True):
    
#     ##check argument validity
#     if out_var != "was_early_preterm" and out_var != "was_early_early_preterm":
#         raise ValueError("outcome must be was_early_preterm or was_early_early_preterm")
    
#     ##obtain data by goal
#     if out_var == "was_early_preterm":
#         data = mydata_early_preterm
#     else:
#         data = mydata_eearly_preterm
        
#     project_list = data['project'].unique()
#      ##accuracy, sen, spe, auc score
#     train_loss = {}
#     val_loss = {}
#     acc = {}
#     sen = {}
#     spe = {}
#     auc = {}
#     precision = {}
#     result_tabs = {}
    
    
#     for project in project_list:
#         train_list = data[data['project']!= project].groupby('participant_id').first().sort_index(axis = 0)
#         train_id = list(train_list.groupby(out_var).sample(frac = train_ratio, random_state = 100).sort_index(axis = 0).index)
#         val_id = np.setdiff1d(list(train_list.index),train_id)
#         test_id = list(data[data['project']== project].groupby('participant_id').first().sort_index(axis = 0).index)
#         train_x, train_y = tensor_generator(data,train_id,features = features, \
#                                             out_var= "was_early_preterm", \
#                                             label_smooth= label_smooth, multi_outcome= multi_outcome)
#         val_x, val_y = tensor_generator(data,val_id,features = features, \
#                                             out_var= "was_early_preterm", \
#                                             label_smooth= label_smooth, multi_outcome= multi_outcome)
        
#         train_dataloader = dataloader(train_x, train_y,batch_size =  batch_size)
#         val_dataloader = dataloader(val_x, val_y,batch_size =  batch_size)
#         model, train_loss[project],val_loss[project] = train_epoch(train_dataloader,val_dataloader, learn_rate = learn_rate, \
#                 hidden_dim = hidden_dim, n_layers = n_layers, drop_prob = drop_prob, \
#                 device = device, EPOCHS = EPOCHS, output_dim = output_dim, method = method)
        
#         acc[project],sen[project],spe[project],auc[project],precision[project],result_tabs[project] = \
#         test_metrics(model, data,test_id,features, out_var = out_var, multi_outcome = multi_outcome)
    
#     return train_loss,val_loss, result_tabs ,pd.DataFrame([acc,sen,spe,precision,auc], \
#                                                  index = ['accuracy','sensitivity','specificity','precision','AUC'])
    
   
    